In [4]:
import sys
sys.path.append("../../../")
import utilssphering
import pandas as pd

In [48]:
# For better display of numbers
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.precision = 3

In [46]:
def top_bottom_features(df: pd.DataFrame, num: int = 10):
    # Drop columns that we don't need
    drop_cols = df.columns[df.columns.str.contains("kstest_statistic|_Location_|Children|Parent|Number_Object")]

    df = df.drop(columns=drop_cols)

    # pvalues are determined between all combinations with replacement, so remove the 
    # identical comparisons
    df = df[df["Assay_Plate_Barcode_plate1"] != df["Assay_Plate_Barcode_plate2"]]

    compartments = ["Nuclei", "Cytoplasm", "Cells"]

    result_df = pd.DataFrame()

    for compart in compartments:
        subset_cols = [col for col in df.columns if compart in col]
        sub_df = df[subset_cols]
        # print(sub_df.columns)

        # Find the first 3 words of column names that will be used for grouping
        group_words = [col.split("_")[:3] for col in subset_cols]
        # Join them back
        group_words = ["_".join(w) for w in group_words]

        # Find the mean for the feature groups
        result_df = pd.concat([result_df, sub_df.groupby(group_words, axis=1).mean().mean(axis=0).reset_index()])

    result_df = result_df.rename(columns={"index": "Feature", 0: "pvalue"})

    return result_df.sort_values(by="pvalue", ascending=False).reset_index(drop=True)[:num], result_df.sort_values(by="pvalue").reset_index(drop=True)[:num]

In [56]:
df = pd.read_csv("../../../data/MolDev_all_plate_comparison_exclude_different_mag_binning_comparison.csv")
top_features, bottom_features = top_bottom_features(df)
t_b_feats = pd.concat([top_features, bottom_features], axis=1)

t_b_feats.to_csv("S8A-moldev-top_bottom_features.csv", index=False, float_format='%f')


In [53]:
df = pd.read_csv("../../../data/Nikon_all_plate_comparison_exclude_different_mag_binning_comparison.csv")
top_features, bottom_features = top_bottom_features(df)
t_b_feats = pd.concat([top_features, bottom_features], axis=1)

t_b_feats.to_csv("S8B-nikon-top_bottom_features.csv", index=False, float_format='%f')

In [54]:
df = pd.read_csv("../../../data/PE_all_plate_comparison_exclude_different_mag_binning_comparison.csv")
top_features, bottom_features = top_bottom_features(df)
t_b_feats = pd.concat([top_features, bottom_features], axis=1)

t_b_feats.to_csv("S8C-pe-top_bottom_features.csv", index=False, float_format='%f')

In [55]:
df = pd.read_csv("../../../data/Yokogawa_US_all_plate_comparison_exclude_different_mag_binning_comparison.csv")
top_features, bottom_features = top_bottom_features(df)
t_b_feats = pd.concat([top_features, bottom_features], axis=1)

t_b_feats.to_csv("S8D-yokogawa_us-top_bottom_features.csv", index=False, float_format='%f')